In [113]:
import pandas as pd
import os
import missingno as msno
import datetime
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.sandbox.regression.predstd import wls_prediction_std

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
wi_zipcode = pd.read_csv("assets\zipcode.csv")
mort_rate = pd.read_csv("assets\mort_rate.csv")

In [ ]:
# Uncomment the following line if the house_price.csv file is not in the assets/house_price folder
# house_price_folder_path = "C:\\Users\\aliso\\Documents\\GitHub\\SIADS-696-Milestone-II\\assets\\house_price"
# os.chdir(house_price_folder_path)

# df = pd.DataFrame([])

# # Read in the data and concatenate into a single dataframe
# for root, dirs, files in os.walk("."):
#     for name in files:
#         df_temp = pd.read_csv(name, encoding = "ISO-8859-1")
#         df = pd.concat([df, df_temp], axis=0)

# # Save the data to a csv file
# df.to_csv("house_price.csv", index=False)

In [2]:
# Read csv files
house_price_df = pd.read_csv("assets\house_price.csv")
house_price_df.head()

C:\Users\aliso\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (47,70,71,72,79) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,OWNER1,OWNER2,Mailing_address1,Mailing_Address2,Mailing_Address3,REAL_ESTATE_ID,CARD_NUMBER,NUMBER_OF_CARDS,Street_Number,Street_Prefix,...,Land_Deferred_code,Land_Deferred_Amount,Historic_Deferred_code,Historic_Deferred_Amount,RECYCLED_UNITS,Disq_and_Qual_flag,Land_Disq_and_Qual_flag,TYPE_AND_USE,PHYSICAL_CITY,PHYSICAL_ZIP_CODE
0,NORTHGATE PROPERTIES LLC,NaN,8465 LENTIC CT,RALEIGH NC 27615-4964,NaN,1,1,1,1506,NaN,...,NaN,0,NaN,0,0,NaN,NaN,NaN,RALEIGH,27604.0
1,BONES 13 LLC,NaN,306 MIDENHALL WAY,CARY NC 27513-5569,NaN,3,1,2,6012,NaN,...,NaN,0,NaN,0,0,A,NaN,20.0,RALEIGH,27617.0
2,BONES 13 LLC,NaN,306 MIDENHALL WAY,CARY NC 27513-5569,NaN,3,2,2,6012,NaN,...,NaN,0,NaN,0,0,A,NaN,34.0,RALEIGH,27617.0
3,HARRISPARK PROPERTIES INC,NaN,608 GASTON ST STE 200,RALEIGH NC 27603-1258,NaN,4,1,1,1601,NaN,...,NaN,0,NaN,0,0,T,NaN,85.0,RALEIGH,27604.0
4,"MOGHADASS, INC",NaN,5040 ISABELLA CANNON DR,RALEIGH NC 27612-4804,NaN,5,1,1,1831,NaN,...,NaN,0,NaN,0,0,A,NaN,200.0,RALEIGH,27604.0


In [3]:
house_price_df.columns

Index(['OWNER1', 'OWNER2', 'Mailing_address1', 'Mailing_Address2',
       'Mailing_Address3', 'REAL_ESTATE_ID', 'CARD_NUMBER', 'NUMBER_OF_CARDS',
       'Street_Number', 'Street_Prefix', 'Street_Name', 'Street_Type',
       'Street_Suffix', 'Planning_Jurisdiction', 'Street_Misc', 'Township',
       'Fire_District', 'Land_Sale_Price', 'Land_Sale_Date', 'Zoning',
       'Deeded_Acreage', 'Total_sale_Price', 'Total_Sale_Date',
       'Assessed_Building_Value', 'Assessed_Land_Value',
       'Parcel_Identification', 'Special_District1', 'Special_District2',
       'Special_District3', 'BILLING_CLASS', 'PROPERTY_DESCRIPTION',
       'Land_classification', 'DEED_BOOK', 'DEED_PAGE', 'Deed_Date', 'VCS',
       'PROPERTY_INDEX', 'Year_Built', 'NUM_of_Rooms', 'UNITS', 'HEATED_AREA',
       'UTILITIES', 'Street_pavement', 'TOPOGRAPHY', 'Year_of_Addition',
       'Effective_year', 'Remodeled_Year', 'Special_Write_In', 'Story_Height',
       'DESIGN_STYLE', 'Foundation_Basement', 'Foundation_Basemen

In [102]:
updated_house_price = house_price_df[['Land_Sale_Price', 'Total_sale_Price', 'Deed_Date', 'Assessed_Building_Value', 'Story_Height', 'HEATED_AREA',
       'UTILITIES', 'Remodeled_Year', 'BATH', 'TYPE_AND_USE', 'PHYSICAL_ZIP_CODE', 'PHYSICAL_CITY']].copy()
updated_house_price['Land_Sale_Price'] = updated_house_price['Land_Sale_Price'].str.replace(',', '').astype(float)
updated_house_price['Total_sale_Price'] = updated_house_price['Total_sale_Price'].str.replace(',', '').astype(float)
updated_house_price['Assessed_Building_Value'] = updated_house_price['Assessed_Building_Value'].str.replace(',', '').astype(float)
updated_house_price['PHYSICAL_ZIP_CODE'] = updated_house_price['PHYSICAL_ZIP_CODE'].fillna(0).astype(int)
updated_house_price['Remodeled_Year'] = pd.to_datetime(updated_house_price['Remodeled_Year'], errors='coerce')
updated_house_price['Deed_Date'] = pd.to_datetime(updated_house_price['Deed_Date'], errors='coerce')


updated_house_price.head()

,Land_Sale_Price,Total_sale_Price,Deed_Date,Assessed_Building_Value,Story_Height,HEATED_AREA,UTILITIES,Remodeled_Year,BATH,TYPE_AND_USE,PHYSICAL_ZIP_CODE,PHYSICAL_CITY
0,0.0,0.0,2020-09-30,0.0,NaN,0,ALL,1970-01-01,NaN,NaN,27604,RALEIGH
1,0.0,740000.0,2012-03-05,524503.0,A,4500,ALL,1970-01-01,I,20.0,27617,RALEIGH
2,0.0,740000.0,2012-03-05,524503.0,A,4008,ALL,1970-01-01,I,34.0,27617,RALEIGH
3,0.0,185000.0,2011-01-19,374128.0,A,8500,ALL,1970-01-01,I,85.0,27604,RALEIGH
4,0.0,750000.0,2016-06-17,238511.0,A,3088,ALL,1970-01-01,I,200.0,27604,RALEIGH


In [103]:
# According to the U.S. Census Bureau, a single-family house is one that may be fully detached, semi-detached, a row house or a townhome. df.loc[df['column_name'].isin(some_values)]
updated_house_price = updated_house_price.loc[updated_house_price['TYPE_AND_USE'].isin([1, 8])]
updated_house_price.loc[updated_house_price['Deed_Date'].isnull(), 'Deed_Date'] = updated_house_price['Remodeled_Year']

In [104]:
updated_house_price = updated_house_price.loc[updated_house_price['Deed_Date'] > "2000-01-01"]
print(f"Total number of zero total sale price is {len(updated_house_price[updated_house_price['Total_sale_Price'] == 0])}")
print(f"Total number of zero land sale price is {len(updated_house_price[updated_house_price['Land_Sale_Price'] == 0])}")
updated_house_price = updated_house_price.loc[updated_house_price['Total_sale_Price'] != 0]


updated_house_price.drop(columns=['Remodeled_Year', ], inplace=True)
updated_house_price.reset_index(drop=True, inplace=True)
updated_house_price

Total number of zero total sale price is 14683
Total number of zero land sale price is 174092


,Land_Sale_Price,Total_sale_Price,Deed_Date,Assessed_Building_Value,Story_Height,HEATED_AREA,UTILITIES,BATH,TYPE_AND_USE,PHYSICAL_ZIP_CODE,PHYSICAL_CITY
0,0.0,37500.0,2004-09-16,118723.0,A,2261,WSE,C,1.0,27591,WENDELL
1,0.0,380000.0,2015-08-12,161077.0,A,1789,ALL,C,1.0,27607,RALEIGH
2,0.0,337500.0,2012-12-27,273621.0,A,2463,ALL,C,1.0,27615,RALEIGH
3,0.0,319000.0,2010-06-21,503301.0,B,4650,ALL,F,1.0,27608,RALEIGH
4,0.0,425000.0,2008-04-15,203178.0,C,1890,ALL,C,1.0,27604,RALEIGH
...,...,...,...,...,...,...,...,...,...,...,...
286828,0.0,1194000.0,2022-08-02,547787.0,L,4520,ALL,F,1.0,27502,APEX
286829,0.0,1364500.0,2022-07-13,560914.0,L,4347,ALL,F,1.0,27502,APEX
286830,0.0,220000.0,2022-06-07,93432.0,A,896,ALL,A,1.0,27529,GARNER
286831,0.0,170000.0,2021-05-03,119201.0,A,896,ALL,A,1.0,27529,GARNER


In [115]:
print(f"The average sale price is ${round(updated_house_price['Total_sale_Price'].mean(), 2)}")
print(f"The average land sale price is ${round(updated_house_price['Land_Sale_Price'].mean(), 2)}")
print(f"The unique values of the story height are {updated_house_price['Story_Height'].value_counts()}")
print(f"The unique values of the utilities are {updated_house_price['BATH'].value_counts()}")

The average sale price is $331495.13
The average land sale price is $26565.62
The unique values of the story height are C    135822
A     84134
B     20624
I     19056
L     12763
K      7129
J      4504
E      1314
D        24
M         3
G         1
O         1
N         1
Name: Story_Height, dtype: int64
The unique values of the utilities are D    120616
F     57063
C     49515
E     28635
J     10444
A     10116
B      8702
I       245
H        26
Name: BATH, dtype: int64
